In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold,StratifiedShuffleSplit

from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import lightgbm as lgb

In [2]:
X=pd.read_csv("Data/x_2.csv", header=None)
Y=pd.read_csv("Data/y_2.csv", header=None)
test_X=pd.read_csv("Data/test_x_2.csv", header=None)

In [3]:
pd.set_option('display.max_columns',None)
Y

,0
0,0
1,0
2,1
3,0
4,0
...,...
13861,0
13862,0
13863,0
13864,0


In [4]:
fit_params={#"early_stopping_rounds":30, 
            "metric" : 'auc', 
            #"eval_set" : [(x_val,y_val)],
            #'categorical_feature': 'auto',
            'random_state':0,
            'deterministic':True,
            'n_jobs':1,
            'num_leaves':27,
            'n_estimators':300,
#             'max_depth':1000,
            'reg_lambda':0.00000,
            'learning_rate':0.05200,
#             'is_unbalance':True,
#             'scale_pos_weight':1.1,
#             'min_child_samples':21,
            'subsample_freq':10,
            'subsample': 0.9101
}

In [5]:
cat_features = [0,1,2,3,4,5,6,7,11,12,13,14,15,128,129,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,155,156,157,171,172,173,174,182]
for c in cat_features:
    if (c+2) not in X:
        continue
    col_type = X[c+2].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c+2] = X[c+2].astype('category')
        test_X[c+2] = test_X[c+2].astype('category')

In [6]:
X.iloc[:,12]=X.iloc[:,12].astype('category')
test_X.iloc[:,12]=test_X.iloc[:,12].astype('category')

In [7]:
clf = lgb.LGBMClassifier(**fit_params)
from sklearn.metrics import log_loss
preds = np.zeros(test_X.shape[0])
n_bag = 100
sss = StratifiedShuffleSplit(n_splits=n_bag, test_size=0.1, random_state=0)
sss.get_n_splits(X, Y)
print(sss)       
ct = 0
bs = []
for train_index, test_index in sss.split(X, Y):
#     break
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train.values.ravel())
    a = (1-log_loss(y_test, clf.predict_proba(X_test)[:,1]))
    if a > 0.57:
    #print(a)
        preds += clf.predict_proba(test_X)[:,1]
        ct+=1
    bs.append(a)

df_describe = pd.DataFrame(bs)
print(df_describe.describe())
print(ct)
scores = 1+cross_val_score(clf, X, Y.values.ravel(), cv = 5, scoring = 'neg_log_loss')
print(scores)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

StratifiedShuffleSplit(n_splits=100, random_state=0, test_size=0.1,
            train_size=None)
                0
count  100.000000
mean     0.565231
std      0.011309
min      0.537692
25%      0.558242
50%      0.565333
75%      0.572275
max      0.594178
35
[0.56478223 0.56029822 0.5731588  0.56090416 0.55924031]
Accuracy: 0.5637 (+/- 0.0102)
